# Import Lib

In [103]:
import sys
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

from sklearn.decomposition import PCA

import open3d as o3d
import alphashape
from descartes import PolygonPatch
from shapely.geometry import Polygon
from shapely import affinity



In [104]:
# switch for figure
show_figure = True

# add aixs
aix_points = [[0, 0, 0],
              [0, 0, 100],
              [400, 0, 0],
              [0, 100, 0],
              [-400, 0, 0],]
aix_lines = [[0, 1], # z-aix
             [0, 2], # x-aix
             [0, 3]] # y-aix

colors = [[1,0,1], [0,0,0], [0,0,0]]
aix_line_set = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(aix_points),
                                    lines=o3d.utility.Vector2iVector(aix_lines))
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

# Read scan model
## import obj file

In [105]:
scan_obj = o3d.io.read_triangle_mesh("../data/femur_half_4.obj")
print(scan_obj)
o3d.visualization.draw_geometries([scan_obj], mesh_show_wireframe=True)

[Open3D WARNING] Read OBJ failed: Cannot open file [../data/femur_half_4.obj]

geometry::TriangleMesh with 0 points and 0 triangles.


RuntimeError: [Open3D ERROR] [CreateCoordinateFrame] size <= 0

## Scale unit length to 1mm (coordinate 1000x)

In [ ]:
points_center = scan_obj.get_center()
print(points_center)
scan_obj.scale(1000.0, points_center)

## change to point cloud

In [ ]:
number_of_points = np.asarray(scan_obj.vertices).shape[0]
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)

In [ ]:
center = scan_pcd.get_center()
center

# --------------------------------------

## delete plane
### find plane using RANSAC: plane function: ax + by + cz + d = 0

In [ ]:
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=2,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
plane = plane_model
print(f"Plane equation: {a:.5f}x + {b:.5f}y + {c:.5f}z + {d:.5f} = 0")

# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

## Delete outliers

In [ ]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=30,
                                                std_ratio=0.5)
# display outliers
display_inlier_outlier(bone_cloud, ind)

In [ ]:
bone_cloud = bone_cloud.select_by_index(ind)

### project point to plane 
link:  \
https://math.stackexchange.com/questions/100761/how-do-i-find-the-projection-of-a-point-onto-a-plane


In [ ]:
def point_to_plane(point, plane):
    A = plane[0]
    B = plane[1]
    C = plane[2]
    D = plane[3]

    xi = point[0]
    yi = point[1]
    zi = point[2]

    t = (A*xi + B*yi + C*zi + D) / (A**2 + B**2 + C**2)
    x = xi - A*t
    y = yi - B*t
    z = zi - C*t
    
    res = [x, y, z]
    return res

In [ ]:
bone_points = np.array(bone_cloud.points)
bone_points

In [ ]:
proj_bone_points = []
for i in range(bone_points.shape[0]):
    point = bone_points[i]
    proj_bone_points.append(point_to_plane(point, plane))

bone_points = np.array(proj_bone_points)
bone_points

In [ ]:
bone_pcd = o3d.geometry.PointCloud()
bone_pcd.points = o3d.utility.Vector3dVector(bone_points)

o3d.visualization.draw_geometries([bone_pcd, aix_line_set])

### PCA


In [ ]:
bone_points = bone_points - bone_points.mean(axis=0, keepdims=True)
print(bone_points)

In [ ]:
pca = PCA(n_components=3)
pca.fit(bone_points)
pca.components_

In [ ]:
x_ = pca.components_[0]
y_ =pca.components_[1]
z_ = np.cross(x_, y_)
tran_matrix = np.array([x_, y_, z_])
tran_matrix


In [ ]:
bone_after_pca = np.dot(tran_matrix, bone_points.T).T
bone_after_pca

In [ ]:
final_pcd = o3d.geometry.PointCloud()
final_pcd.points = o3d.utility.Vector3dVector(bone_after_pca)
o3d.visualization.draw_geometries([final_pcd, aix_line_set])


## save file

In [ ]:
o3d.io.write_point_cloud("./data/2d_bone.ply", final_pcd)


In [ ]:
type(bone_after_pca)

In [ ]:
points = []
text_file = open("../data/2dpoints.txt", "w")
for i in range(bone_after_pca.shape[0]):
    point = bone_after_pca[i]
    x = point[0]
    y = point[1]
    points.append([x, y])
    content = str(x) + ',' + str(y) + '\n'
    text_file.write(content)
text_file.close()


In [ ]:
points = np.array(points)



In [ ]:
# todo
alpha_shape = alphashape.alphashape(points, 0.4)
# fig, ax = plt.subplots()
# ax.scatter(points[:, 0], points[:,1])
# ax.add_patch(PolygonPatch(alpha_shape, alpha=1))
# plt.show()

alpha_shape


In [ ]:
# todo: up and down flip
(minx, miny, maxx, maxy) = alpha_shape.exterior.bounds

x_length = maxx - minx
y_length = maxy - miny
center_box =  Polygon([(minx/5, miny), (minx/5, maxy), (maxx/5, maxy), (maxx/5, miny)])
center_bone = alpha_shape.intersection(center_box)

if (maxy - center_bone.centroid.y) > (maxy - miny) / 2:
    alpha_shape = affinity.scale(alpha_shape, xfact=1, yfact=-1, origin=(0,0))
alpha_shape

In [ ]:
# left and right flip
left_box = Polygon([(minx,miny), (minx, maxy), (minx + x_length/10,maxy), (minx + x_length/10, miny)])
left_bone = alpha_shape.intersection(left_box)

right_box =  Polygon([(maxx - x_length/10, miny), (maxx - x_length/10, maxy), (maxx, maxy), (maxx, miny)])
right_bone = alpha_shape.intersection(right_box)

if left_bone.area <  right_bone.area:
    alpha_shape = affinity.scale(alpha_shape, xfact=-1, yfact=1, origin=(0,0))
alpha_shape

In [ ]:
alpha_shape.boundary

In [ ]:
type(alpha_shape.boundary)

In [ ]:
alpha_shape.boundary.bounds


In [ ]:
line = alpha_shape.exterior
type(line)


## Measurement

### FML

In [ ]:
(minx, miny, maxx, maxy) = alpha_shape.exterior.bounds
print(minx, miny, maxx, maxy)
fml = maxx - minx
print('fml: ', fml)

### FEB


In [ ]:
# todo
left_box = Polygon([(minx,miny), (minx, maxy), (minx/1.5,maxy), (minx/1.5, miny)])
left_bone = alpha_shape.intersection(left_box)
left_bone

In [ ]:
(left_bone_minx, left_bone_miny, left_bone_maxx, left_bone_maxy) = left_bone.exterior.bounds
print(left_bone_minx, left_bone_miny, left_bone_maxx, left_bone_maxy)
feb = left_bone_maxy - left_bone_miny
print('feb: ', feb)

### FBML

In [ ]:
left_bone_line = left_bone.exterior

In [ ]:
left_bone.boundary

In [ ]:
left_bone_points = []
for x,y in left_bone_line.coords:
    left_bone_points.append([x, y])
left_bone_points = left_bone_points[:-1]
len(left_bone_points)

In [ ]:
left_bone_height = left_bone_maxy - left_bone_miny

In [ ]:
max_diff_y = 0
a_idx = 0
for i in range(len(left_bone_points) - 1):
    diff_cur = abs(left_bone_points[i][1] - left_bone_points[i+1][1])
    if  diff_cur > max_diff_y:
        max_diff_y = diff_cur
        a_idx = i
print(a_idx)
print(max_diff_y)
point_a = left_bone_points[a_idx]
point_b = left_bone_points[a_idx + 1]

print(point_a, point_b)

In [ ]:
left_bone_points_ordered = left_bone_points[a_idx+1:] + left_bone_points[:a_idx+1]
len(left_bone_points_ordered)

In [ ]:
p_left = []
p_left_idx = 0
for i in range(len(left_bone_points_ordered)):
    if left_bone_points_ordered[i][0] == left_bone_minx:
        p_left = left_bone_points_ordered[i]
        p_left_idx = i
        break
print(p_left, 'at: ', p_left_idx)

In [ ]:
# in upper part
if left_bone_maxy - p_left[1] < (left_bone_maxy - left_bone_miny) * 0.5:
    left_bone_points_ordered.reverse()
    p_left_idx = len(left_bone_points_ordered) - 1 - p_left_idx

left_bone_points_ordered = left_bone_points_ordered[p_left_idx+1:]
x_min_point_index = 0

# 如果最小的点不是线段头上的前十个点，说明它是P2
while x_min_point_index < 10:
    # 去掉B到current_point， 剩下所有的点里面 最左侧的点找出来
    left_bone_points_ordered = left_bone_points_ordered[x_min_point_index+1:]
    x_min_point = min(left_bone_points_ordered, key = lambda t: t[0])
    x_min_point_index = [i for i, j in enumerate(left_bone_points_ordered) if j == x_min_point][0]

In [ ]:
p_left_second = x_min_point
print('p_left_second: ', p_left_second)


In [ ]:
# todo
right_box =  Polygon([(maxx/1.5, miny), (maxx/1.5, maxy), (maxx, maxy), (maxx, miny)])
right_bone = alpha_shape.intersection(right_box)
right_bone

In [ ]:
(right_bone_minx, right_bone_miny, right_bone_maxx, right_bone_maxy) = right_bone.exterior.bounds
print(right_bone_minx, right_bone_miny, right_bone_maxx, right_bone_maxy)

In [ ]:
right_bone_line = right_bone.exterior

In [ ]:
right_bone.boundary

In [ ]:
right_bone_points = []
for x,y in right_bone_line.coords:
    right_bone_points.append([x, y])
right_bone_points = right_bone_points[:-1]
len(right_bone_points)

In [ ]:
max_diff_y = 0
a_idx = 0
for i in range(len(right_bone_points) - 1):
    diff_cur = abs(right_bone_points[i][1] - right_bone_points[i+1][1])
    if  diff_cur > max_diff_y:
        max_diff_y = diff_cur
        a_idx = i
print(a_idx)
print(max_diff_y)
point_a = right_bone_points[a_idx]
point_b = right_bone_points[a_idx + 1]

print(point_a, point_b)

In [ ]:
right_bone_points_ordered = right_bone_points[a_idx+1:] + right_bone_points[:a_idx+1]
len(right_bone_points_ordered)

In [ ]:
def distance_point_to_line(p1, p2, p3):
    # point1 and point 2 formed a line, dis is point to that line
    p1 = np.asarray(p1)
    p2 = np.asarray(p2)
    p3 = np.asarray(p3)
    return np.abs(np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1))

fbml = 0
for i in range (len(right_bone_points_ordered)):
    fbml = max(fbml, distance_point_to_line(p_left, p_left_second, right_bone_points_ordered[i]))

print('fbml: ', fbml)


### FMLD

In [ ]:
# todo
center_box =  Polygon([(minx/3, miny), (minx/3, maxy), (maxx/3, maxy), (maxx/3, miny)])
center_bone = alpha_shape.intersection(center_box)
center_bone

In [ ]:
(center_bone_minx, center_bone_miny, center_bone_maxx, center_bone_maxy) = center_bone.exterior.bounds
print(center_bone_minx, center_bone_miny, center_bone_maxx, center_bone_maxy)

In [ ]:
center_bone_line = center_bone.exterior

In [ ]:
center_bone.boundary

In [ ]:
center_bone_points = []
for x,y in center_bone_line.coords:
    center_bone_points.append([x, y])
center_bone_points = center_bone_points[:-1]
len(center_bone_points)

In [ ]:
center_bone_points = np.asarray(center_bone_points)

In [ ]:
import numpy.polynomial.polynomial as poly

center_bone_points_upper = np.asarray([x for x in center_bone_points if x[1] >= 0])
center_bone_points_lower = np.asarray([x for x in center_bone_points if x[1] <= 0])

In [ ]:
def fit_line(points_array):
    x = points_array[:, 0]
    y = points_array[:, 1]
    x_new = np.linspace(x[0], x[-1], num=len(x)*10)

    coefs = poly.polyfit(x, y, 2)
    ffit = poly.polyval(x_new, coefs)

    y_draw = []
    for num in x_new:
        y_draw.append(coefs[2] * num*num + coefs[1] * num + coefs[0])

    if show_figure:
        plt.plot(x, y, label = "line 1")
        #plt.plot(x_new, ffit, label = "line 2")
        plt.plot(x_new, y_draw, label="line 3")

        plt.legend()
        # plt.axes().set_aspect(1)
        plt.show()
    return coefs

# j = coefs_up[3] * (i**3) + coefs_up[2] * (i**2) + coefs_up[1] * i + coefs_up[0]
top_line_p = fit_line(center_bone_points_upper)
bottom_line_p = fit_line(center_bone_points_lower)
print(top_line_p, bottom_line_p)

In [ ]:
if show_figure:
    x = np.linspace(-25, 25, num=100)
    a = top_line_p[2] * x * x + top_line_p[1] * x + top_line_p[0]
    b = bottom_line_p[2] * x * x + bottom_line_p[1] * x + bottom_line_p[0]
    plt.plot(x, a, 'r') # plotting t, a separately
    plt.plot(x, b, 'b') # plotting t, b separately
    plt.plot([0], [0], '*') # plotting t, c separately
    plt.show()

In [ ]:
# vertical line
fmld = poly.polyval(0, top_line_p) - poly.polyval(0, bottom_line_p)
print(fmld)

In [ ]:
def distance_2_point_to_point(p1, p2):
    return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2

min_line_segment_length = fmld **2
for i in np.arange(-20,20,.01):
    if i == 0:
        continue
    x = i
    y = top_line_p[2] * x * x + top_line_p[1] * x + top_line_p[0]
    k = y/x
    [x_res1, x_res2] = np.roots([bottom_line_p[2], bottom_line_p[1] - k, bottom_line_p[0]])

    y_res1 = k * x_res1
    dis_1 = x_res1 **2 + y_res1**2

    y_res2 = k * x_res2
    dis_2 = x_res2 **2 + y_res2**2

    [x1, y1] = [x_res1, y_res1] if dis_1 < dis_2 else [x_res2, y_res2]

    dis_cur = distance_2_point_to_point([x, y], [x1, y1])
    min_line_segment_length = min(dis_cur, min_line_segment_length)

fmld = math.sqrt(min_line_segment_length)
print('fmld: ', fmld)


### FHD

In [ ]:
#1. 假设骨头的一圈点是个list，这个list是顺时针记录的骨头的点，起点是左上角
# for i in right_bone_points_ordered:
#     print(i)

In [ ]:
#2. 找到骨头最右侧的点 max(x)
(right_bone_minx, right_bone_miny, right_bone_maxx, right_bone_maxy) = right_bone.exterior.bounds
print(right_bone_minx, right_bone_miny, right_bone_maxx, right_bone_maxy)

right_most_idx = 0
for i in range(len(right_bone_points_ordered)):
    if right_bone_points_ordered[i][0] == right_bone_maxx:
        right_most_idx = i
        break
right_most_idx

In [ ]:
#3. 以该点向上五个点开始 找到起点 start_point
start_idx = right_most_idx - 5

In [ ]:
#4. 找出start_point代表的斜率为1的曲线tmp_line start_point_y = start_point_x+intercept
start_x = right_bone_points_ordered[start_idx][0]
start_y = right_bone_points_ordered[start_idx][1]
print(start_x, start_y)

# line
intercept = start_y - start_x
intercept

In [ ]:
#5. 从start_point开始 向下遍历点 找y=x+b 和弧线的另一个交点
# line p1: (start_x, start_y), p2:(0, intercept), p3: tmp_point2

p_start = [start_x, start_y]
p_y_inter = [0, intercept]

# right: True; left : False
side = True
end_idx = start_idx + 3
while end_idx < len(right_bone_points_ordered):
    cur_x = right_bone_points_ordered[end_idx][0]
    cur_y = right_bone_points_ordered[end_idx][1]

    if cur_x + intercept > cur_y:
        side = True
    else:
        side = False

    if not side:
        break
    end_idx += 1

line_right_point = [right_bone_points_ordered[end_idx - 1][0], right_bone_points_ordered[end_idx - 1][1]]
line_left_point = [right_bone_points_ordered[end_idx][0], right_bone_points_ordered[end_idx][1]]
print(line_left_point, line_right_point)

dis_right_point = distance_point_to_line(p_start, p_y_inter, line_right_point)
dis_left_point = distance_point_to_line(p_start, p_y_inter, line_left_point)

print(dis_left_point, dis_right_point)

p_start_2 = line_left_point
p_start_2_idx = end_idx
if dis_left_point > dis_right_point:
    p_start_2 = line_right_point
    p_start_2_idx = end_idx - 1

print('start: ', p_start)
print('start2: ', p_start_2)


In [ ]:
#6. 计算出点tmp_point2到 tmp_line上的投影点tmp_point3 然后算出来 start_point到tmp_point3的距离dist
# 把dist存入一个list
def distance_point_to_point(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

fhd = distance_point_to_point(p_start, p_start_2)
count_decrease = 0
pre_fhd = fhd
pre_down_left = p_start_2
pre_down_left_idx = p_start_2_idx

In [ ]:
#7. start_point开始，向上遍历点 tmp_point1， 更新tmp_line   tmp_point1_y = tmp1_point_x+b
#8. 每次遍历，找到在上次tmp_point2附近的一个新点，使得新点到直线tmp_line距离最短
# tmp_point2更新为新的点，然后计算tmp_point2在直线tmp_line上的投影tmp_point3 到 tmp_point1的距离，存入dist list
# 依次循环,dist应该先增大，后减小，我们要找到的就是增到极大值，开始减小时候的那个极大的距离

def find_down_left(p1, from_idx, right_bone_points_ordered_array):
    cur_intercept = p1[1] - p1[0]
    right_side = True
    while right_side:
        candidate_x = right_bone_points_ordered_array[from_idx][0]
        candidate_y = right_bone_points_ordered_array[from_idx][1]
        if candidate_x + cur_intercept <= candidate_y:
            right_side = False
        from_idx += 1

    left_idx = from_idx - 1
    right_idx = left_idx - 1
    left_dis = distance_point_to_line(p1, [0, cur_intercept], right_bone_points_ordered_array[left_idx])
    right_dis = distance_point_to_line(p1, [0, cur_intercept], right_bone_points_ordered_array[right_idx])

    res_point = right_bone_points_ordered_array[left_idx]
    if left_dis > right_dis:
        res_point = right_bone_points_ordered_array[right_idx]

    return res_point


iterate_idx = start_idx
while iterate_idx > 0:
    p_up_right = right_bone_points_ordered[iterate_idx]

    p_down_left = find_down_left(p_up_right, p_start_2_idx, right_bone_points_ordered)
    cur_fhd = distance_point_to_point(p_up_right, p_down_left)
    fhd = max(fhd, cur_fhd)
    if cur_fhd < fhd:
        count_decrease += 1
        if count_decrease >= 5:
            break
    else:
        count_decrease = 0

    pre_fhd = cur_fhd
    iterate_idx -= 1

print('fhd: ', fhd)
